# Sentimental analysis on customers review

In [1]:
# Import libs
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt 
from textblob import TextBlob

In [2]:
# Read df of products
df_prod = pd.read_csv('https://raw.githubusercontent.com/r41ss4/rennes_ds/refs/heads/main/data/raw/raw_products.csv')
df_prod.head(5)

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


In [3]:
# Review columns
df_prod.columns

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [4]:
df_prod[['product_id', 'product_name', 'loves_count', 'rating', 'reviews']].head()

,product_id,product_name,loves_count,rating,reviews
0,P473671,Fragrance Discovery Set,6320,3.6364,11.0
1,P473668,La Habana Eau de Parfum,3827,4.1538,13.0
2,P473662,Rainbow Bar Eau de Parfum,3253,4.2500,16.0
3,P473660,Kasbah Eau de Parfum,3018,4.4762,21.0
4,P473658,Purple Haze Eau de Parfum,2691,3.2308,13.0


In [5]:
# Read df of products
df_rev = pd.read_csv('https://raw.githubusercontent.com/r41ss4/rennes_ds/refs/heads/main/data/raw/raw_reviews.csv')
df_rev.head(5)

/var/folders/q5/bwsxp4bx3jvb0n9scz0gsk500000gn/T/ipykernel_78812/607858644.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rev = pd.read_csv('https://raw.githubusercontent.com/r41ss4/rennes_ds/refs/heads/main/data/raw/raw_reviews.csv')


,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,I absolutely L-O-V-E this oil. I have acne pro...,A must have!,lightMedium,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
1,1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,I gave this 3 stars because it give me tiny li...,it keeps oily skin under control,mediumTan,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
2,2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,Works well as soon as I wash my face and pat d...,Worth the money!,lightMedium,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
3,3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,"this oil helped with hydration and breakouts, ...",best face oil,lightMedium,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
4,4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,This is my first product review ever so that s...,Maskne miracle,mediumTan,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0


In [6]:
# Eliminate column Unnamed: 0
df_rev.drop('Unnamed: 0', axis=1, inplace=True)
# Review changes
df_rev.columns

Index(['author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')

In [7]:
# Reviews general info 
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49977 entries, 0 to 49976
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   author_id                 49977 non-null  object 
 1   rating                    49977 non-null  int64  
 2   is_recommended            46160 non-null  float64
 3   helpfulness               36522 non-null  float64
 4   total_feedback_count      49977 non-null  int64  
 5   total_neg_feedback_count  49977 non-null  int64  
 6   total_pos_feedback_count  49977 non-null  int64  
 7   submission_time           49977 non-null  object 
 8   review_text               49918 non-null  object 
 9   review_title              35599 non-null  object 
 10  skin_tone                 42776 non-null  object 
 11  eye_color                 43717 non-null  object 
 12  skin_type                 46346 non-null  object 
 13  hair_color                41126 non-null  object 
 14  produc

In [8]:
# Rename columns for practicity
df_rev.rename(columns = {'review_text':'text'}, inplace = True)
df_rev.head()

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,I absolutely L-O-V-E this oil. I have acne pro...,A must have!,lightMedium,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,I gave this 3 stars because it give me tiny li...,it keeps oily skin under control,mediumTan,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,Works well as soon as I wash my face and pat d...,Worth the money!,lightMedium,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,"this oil helped with hydration and breakouts, ...",best face oil,lightMedium,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0
4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,This is my first product review ever so that s...,Maskne miracle,mediumTan,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0


In [9]:
# Subset data
df_rev_s = df_rev[['text','is_recommended', 'rating', 'product_id', 'product_name', 'brand_name']]
# Review 
df_rev_s.head()

,text,is_recommended,rating,product_id,product_name,brand_name
0,I absolutely L-O-V-E this oil. I have acne pro...,1.0,5,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins
1,I gave this 3 stars because it give me tiny li...,1.0,3,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins
2,Works well as soon as I wash my face and pat d...,1.0,5,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins
3,"this oil helped with hydration and breakouts, ...",1.0,5,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins
4,This is my first product review ever so that s...,1.0,5,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins


In [10]:
df_rev_s['is_recommended'].value_counts()

is_recommended
1.0    37543
0.0     8617
Name: count, dtype: int64

In [11]:
# Calculate percentage of is_recommended True
label_pos = (df_rev_s['is_recommended'].value_counts()[1] / len(df_rev_s) *100).round(2)
label_pos

75.12

In [12]:
# Calculate percentage of is_recommended False
label_neg = (df_rev_s['is_recommended'].value_counts()[0] / len(df_rev_s) *100).round(2)
label_neg

17.24

In [13]:
# Print percentage of is_recommended 
print("Positive is_recommended percentage", label_pos, "%")
print("Negative is_recommended percentage", label_neg, "%")

Positive is_recommended percentage 75.12 %
Negative is_recommended percentage 17.24 %


## Text Cleaning 
Symbols, spaces and other uncommon text expression can complicate sentimental analysis. The reviews must first be cleaning or prepare to avoid such problems.

In [14]:
# Import libs
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem import PorterStemmer

In [15]:
def preprocess_text(text, remove_digits=True):
    # Ensure the input is a string
    if not isinstance(text, str):
        text = str(text)
    
    # Removing HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Removing square brackets
    text = re.sub(r'\[[^]]*\]', '', text)
    
    # Removing special characters
    if remove_digits:
        text = re.sub(r'[^a-zA-Z\s]', '', text)
    else:
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Lowercasing
    text = text.lower()
    
    # Stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    
    # Removing stopwords
    stopword_list = set(stopwords.words('english'))
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

In [16]:
# Print original review
print('Before preprocessing \n', df_rev['text'][3])

Before preprocessing 
 this oil helped with hydration and breakouts, I love this!!


In [17]:
# Store review and clean it
df_rev['text'] = df_rev['text'].apply(preprocess_text)

/var/folders/q5/bwsxp4bx3jvb0n9scz0gsk500000gn/T/ipykernel_78812/1560165754.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [18]:
print('After preprocessing \n', df_rev_s['text'][3])

After preprocessing 
 this oil helped with hydration and breakouts, I love this!!


## Polarity based on reviews
This analysis is based on determining whether the sentiment is positive, negative, or neutral is straightforward with libraries like TextBlob or VADER.

In [19]:
# Define a function to get polarity
def get_polarity(text):
    analysis = TextBlob(str(text))
    return analysis.sentiment.polarity

In [20]:
# Apply the function to create a polarity column
df_rev['polarity'] = df_rev['text'].apply(get_polarity)

# Display the updated DataFrame with the new polarity column
df_rev.head()

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,polarity
0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,absolut love thi oil acn prone skin use thi to...,A must have!,lightMedium,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.003333
1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,gave thi star becaus give tini littl white hea...,it keeps oily skin under control,mediumTan,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.083333
2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,work well soon wash face pat dri smell amaz,Worth the money!,lightMedium,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.000000
3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,thi oil help hydrat breakout love thi,best face oil,lightMedium,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.500000
4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,thi first product review ever tell much love t...,Maskne miracle,mediumTan,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.127083


In [21]:
# Get max of polarity
max_pol = df_rev['polarity'].max()
# Show result
max_pol

1.0

In [22]:
# Get min of polarity
min_pol = df_rev['polarity'].min()
# Show result
min_pol

-1.0

In [23]:
# Define the bins and labels
bins = [-1.1,  -0.5,   0,   0.5,  1.1]
labels = [   1,    2,   3,     4]

In [24]:
# Create the 'polarity_level' column based on the bins and labels
df_rev['polarity_level'] = pd.cut(df_rev['polarity'], 
                                bins=bins, 
                                labels=labels)
df_rev.head()

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,polarity,polarity_level
0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,absolut love thi oil acn prone skin use thi to...,A must have!,lightMedium,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.003333,3
1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,gave thi star becaus give tini littl white hea...,it keeps oily skin under control,mediumTan,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.083333,3
2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,work well soon wash face pat dri smell amaz,Worth the money!,lightMedium,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.000000,2
3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,thi oil help hydrat breakout love thi,best face oil,lightMedium,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.500000,3
4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,thi first product review ever tell much love t...,Maskne miracle,mediumTan,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.127083,3


## Intensity analysis
We use the VADER sentiment analyzer to calculate the compound score for each review, indicating the intensity of the sentiment.

In [25]:
# Instal vaderSentiment if needed
!pip install vaderSentiment

In [26]:
# Import libs
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [27]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [28]:
# Function to calculate sentiment intensity
def calculate_intensity(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

In [29]:
# Apply intensity analysis
df_rev['intensity'] = df_rev['text'].apply(calculate_intensity)

In [30]:
# Review new column
df_rev.head(8)

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,text,review_title,...,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,polarity,polarity_level,intensity
0,1945004256,5,1.0,0.000000,2,2,0,2022-12-10,absolut love thi oil acn prone skin use thi to...,A must have!,...,green,combination,NaN,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.003333,3,0.7964
1,5478482359,3,1.0,0.333333,3,2,1,2021-12-17,gave thi star becaus give tini littl white hea...,it keeps oily skin under control,...,brown,oily,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.083333,3,-0.3182
2,29002209922,5,1.0,1.000000,2,0,2,2021-06-07,work well soon wash face pat dri smell amaz,Worth the money!,...,brown,dry,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.000000,2,0.2732
3,7391078463,5,1.0,1.000000,2,0,2,2021-05-21,thi oil help hydrat breakout love thi,best face oil,...,brown,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.500000,3,0.7845
4,1766313888,5,1.0,1.000000,13,0,13,2021-03-29,thi first product review ever tell much love t...,Maskne miracle,...,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.127083,3,0.8176
5,1532951999,5,1.0,1.000000,5,0,5,2020-09-20,use thi year complaint keep hormon skin glow b...,Best Oil,...,brown,normal,brown,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.062500,3,0.5574
6,2168991896,5,1.0,0.900000,10,1,9,2020-08-20,first use lotu oil howev dure move unfortun br...,Shame on me for waiting this long...,...,brown,combination,black,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.047727,3,-0.5574
7,5240639417,5,1.0,0.933333,15,1,14,2020-08-01,super short back stori wa teen mother work cla...,My favorite for many years,...,blue,combination,blonde,P379064,Lotus Balancing & Hydrating Natural Face Treat...,Clarins,65.0,0.290278,3,0.9531


In [31]:
# Get max of intensity
max_int = df_rev['intensity'].max()
# Show result
max_int

0.9981

In [32]:
# Get min of intensity
min_int = df_rev['intensity'].min()
# Show result
min_int

-0.9682

## Aspect-Based Sentiment Analysis
Use textblob (as there are version problems with spacy) to identify aspects (e.g., product features) and then analyze the sentiment associated with each aspect. The new column aspects is supposed to contains tuples representing the aspects (features or topics) mentioned in each review and their associated sentiment scores. 

In [33]:
# Import libs & download nltk content
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lexi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /Users/lexi/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lexi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lexi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
# Function to extract aspects and their sentiments using TextBlob
def extract_aspects(text):
    blob = TextBlob(text)
    aspects = []
    for sentence in blob.sentences:
        for word, tag in sentence.tags:
            if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
                aspect = word
                sentiment = sentence.sentiment.polarity
                aspects.append((aspect, sentiment))
    return aspects

In [ ]:
# Apply aspect extraction
df_rev['aspects'] = df_rev['text'].apply(extract_aspects)

In [ ]:
# Review new column
df_rev.head(8)